<a href="https://colab.research.google.com/github/susandong/w266_final_project_game_sentiment/blob/master/w266_Final_Project_Game_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project: Game Review Sentiment Analysis Over Time
## Research Question: 
* Can we use sentiment analysis score to predict the active user base for video games over time

## Dataset: 
* Game Review: twitter/reddit/discord/steam reviews
* active user base: steam

## Algorithm: 
* Baseline(logistic Regression); 
* Transformer(Elmo/Bert)


In [1]:
#Load libraries
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
""" Download Data: There are 4 games with review data
Fall Guys (fg)
PlayerUnknown Battlegrounds (pubg)
Dota 2 (dota2)
Counterstrike Source: Go (csgo)

Review data has the following columns:
app: ID for the game
useful: how many users voted the review as useful
funny: how many users voted the review as funny
username: username of the person who wrote the review
games_owned: how many games the reviewer owns on Steam
num_reviews: how many reviews the reviewer has written on Steam
recommend: 1 for recommend (thumbs up), -1 for do not recommend (thumbs down)
hours_played: number of hours the reviewer played before writing the review
date: date review was written
text: text of the review
"""
#Fall Guys
fg_url = 'https://raw.githubusercontent.com/susandong/w266_final_project_game_sentiment/master/data/fallguys_reviews.csv'
fg_df = pd.read_csv(fg_url, error_bad_lines=False)
fg_df = fg_df.dropna()

#CS: Go
csgo_url = 'https://raw.githubusercontent.com/susandong/w266_final_project_game_sentiment/master/data/csgo_reviews.csv'
csgo_df = pd.read_csv(csgo_url, error_bad_lines=False)
csgo_df = csgo_df.dropna()

#PUBG
pubg_url = 'https://raw.githubusercontent.com/susandong/w266_final_project_game_sentiment/master/data/pubg_reviews.csv'
pubg_df = pd.read_csv(pubg_url, error_bad_lines=False)
pubg_df = pubg_df.dropna()

#dota2_url = 'https://raw.githubusercontent.com/susandong/w266_final_project_game_sentiment/master/data/dota2_reviews.csv'
#dota2_df = pd.read_csv(dota2_url, error_bad_lines=False)
#player_url = 'https://raw.githubusercontent.com/susandong/w266_final_project_game_sentiment/master/data/PlayerCountData.csv'
#player_df = pd.read_csv(player_url, error_bad_lines=False)

In [3]:
# Data Preprocessing
!pip install unidecode
import unidecode
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
#from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

tknzr = TweetTokenizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
#lemma = WordNetLemmatizer()

#Convert accented characters
def remove_accents(text):
  try:
    text = unidecode.unidecode(text)
  except:
    pass
  return text

#Remove digits and punctuation
def remove_nonletters(text):
  try:
    #Remove digits AND punctuation
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    #Remove just digits that are by themselves
    #text = re.sub('^\d+\s|\s\d+\s|\s\d+$', ' ', text)
  except:
    pass
  return text


#Use Tweet Tokenizer for some built-in emoji support
def tweet_tokenization(text):
  try:
    return tknzr.tokenize(text)
  except:
    pass

def remove_stopwords(token):
  try:
    return [item for item in token if item not in stop_words]
  except:
    pass

def remove_stemmer(token):
  try:
    return [stemmer.stem(i) for i in token]
  except:
    pass

def remove_lemmatizer(token):
  try:
    return [lemma.lemmatize(word=w, pow='v') for w in token]
  except:
    pass
  
def remove_shortwords(token):
  try:
    return [i for i in token if len(i) > 1]
  except:
    pass

def token_to_string(listTokens):
  return ' '.join(listTokens)

#Process text from dataframe. df = dataframe to clean, text = name of column with text
def process_text(df, text):  
  #Create new column for cleaned text
  df['cleaned'] = df[text]

  #Lower case all text
  df['cleaned'] = df['cleaned'].str.lower()

  #Clean URLs
  df['cleaned'] = df['cleaned'].str.replace('http\S+|www.\S+', '', case=False)

  #Remove accents from text
  #df['cleaned'] = df['cleaned'].apply(remove_accents)

  #Remove numbers and punctuation from text
  df['cleaned'] = df['cleaned'].apply(remove_nonletters)
  
  #Tokenize
  df['cleaned'] = df['cleaned'].apply(tweet_tokenization)

  #Remove stopwords
  df['cleaned'] = df['cleaned'].apply(remove_stopwords)
  
  #Remove short words
  df['cleaned'] = df['cleaned'].apply(remove_shortwords)

  #Stemming - can decide to use or not
  #df['cleaned'] = df['cleaned'].apply(remove_stemmer)

  #Convert tokens back to string
  df['cleaned'] = df['cleaned'].apply(token_to_string)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#preprocess all the datasets for all 3 games
process_text(fg_df, 'text')
#len(fg_df['cleaned'][11]) < 2
process_text(csgo_df, 'text')
process_text(pubg_df, 'text')

In [5]:
fg_df

,app,useful,funny,username,games_owned,num_reviews,recommend,hours_played,date,text,cleaned
0,1097150,0,0,7.65612E+16,51,16,1,17.1,"11 October, 2020",ow i fell:( thats a sad face btwincase you did...,ow fell thats sad face btwincase didnt kno whelp
1,1097150,0,0,7.65612E+16,1,1,1,50.4,"11 October, 2020",yes,yes
2,1097150,0,0,7.65612E+16,64,3,-1,8.1,"11 October, 2020",This Game is not fun. If your looking for a ga...,game fun looking game frustrate anger little r...
3,1097150,0,0,floolp,1,1,1,15.3,"11 October, 2020",Fun but VERY HARD game!this is a very fun game...,fun hard game fun game get lots character cust...
4,1097150,0,0,7.65612E+16,6,1,1,34.9,"11 October, 2020",its fun,fun
...,...,...,...,...,...,...,...,...,...,...,...
90653,1097150,3,0,7.65612E+16,0,1,-1,2.7,"8 September, 2020",A TOTAL GARBAGE!!i wish i never bought it and ...,total garbage wish never bought want money bac...
90654,1097150,0,0,quazwaz,15,6,1,16.5,"8 September, 2020",this game is so simple my mom likes it 10/10,game simple mom likes
90655,1097150,1,0,Elbourne,407,1,-1,18.7,"8 September, 2020","ruined by hackers and refused a refund, avoid.",ruined hackers refused refund avoid
90656,1097150,0,0,7.65612E+16,8,1,1,14.7,"8 September, 2020",very good game and fun to play,good game fun play


In [6]:
#concatenate all 3 games data to one large dataset
all_df=pd.concat([fg_df,csgo_df,pubg_df])
all_clean_df=all_df[["recommend","cleaned"]]


In [7]:
from nltk.tokenize import RegexpTokenizer
def remove_blankrow(df,column):
  tokenizer = RegexpTokenizer(r'\w+')
  df['lens'] =[len(tokenizer.tokenize(utterance)) for utterance in df[column]]
  df_clean=df[df['lens'] !=0]
  return df_clean

In [8]:
df_final=remove_blankrow(all_clean_df,'cleaned')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
df_final.shape

(431037, 3)

In [10]:
all_clean_df.shape

(452858, 3)

In [11]:
#check the length of reviews
print(df_final.lens.describe())
print(df_final['lens'].quantile(0.999))
#remove the extremely long reviews
df_final=df_final[df_final.lens<(df_final['lens'].quantile(0.999))]

count    431037.000000
mean         13.906363
std          33.503735
min           1.000000
25%           2.000000
50%           4.000000
75%          12.000000
max        2667.000000
Name: lens, dtype: float64
398.0


In [12]:
#check negative data vs positive data
df_final.recommend.value_counts()

 1    314533
-1    116070
Name: recommend, dtype: int64

In [13]:
#balance the negative and positive data
from sklearn.utils import resample

df_majority = df_final[df_final.recommend==1]
df_minority = df_final[df_final.recommend==-1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=314533,    # to match majority class
                                 random_state=235) # reproducible results
 
# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])
 

#replace outcome label -1 with 0
df_balanced['recommend'] = df_balanced['recommend'].replace([-1],0)
# Display new class counts
df_balanced.recommend.value_counts()

1    314533
0    314533
Name: recommend, dtype: int64

In [14]:
# Build model

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced.cleaned,df_balanced.recommend,
test_size=0.2)


In [16]:
#Bag of Words + XGBoost
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [19]:
#Start with Bag of Words
vectorizer = CountVectorizer(analyzer = "word", max_features=1500, min_df=.01, max_df=0.7) 
bow_train = vectorizer.fit_transform(X_train).toarray()

In [25]:
#XGB classifier
BOW_XGB = xgb.XGBClassifier(max_depth=7, n_estimators=300, objective="binary:logistic", 
                            random_state=1, tree_method='gpu_hist', predictor='gpu_predictor')
BOW_XGB_scores = cross_val_score(BOW_XGB, bow_train, y_train, cv=3, n_jobs=-1)
print("Averaged CV Accuracy: %0.5f" % BOW_XGB_scores.mean())

BOW_XGB.fit(bow_train, y_train)


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Averaged CV Accuracy: 0.80831


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic',
              predictor='gpu_predictor', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, tree_method='gpu_hist', verbosity=1)

In [23]:
#bow_test = (vectorizer.transform(X_test)).toarray()
XGB_pred = BOW_XGB.predict(bow_test)

In [24]:
BOW_XGB_scores_test = cross_val_score(BOW_XGB, bow_test, y_test, cv=3, n_jobs=-1)
print("Averaged CV Accuracy: %0.5f (+/- %0.5f)" % (BOW_XGB_scores_test.mean(), BOW_XGB_scores_test.std() * 2))

Averaged CV Accuracy: 0.80168 (+/- 0.00403)


In [14]:
# Simple CNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np


max_features = 200000  # Only consider the top 20k words
maxlen = 500
embedding_dim = 16


In [16]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
layer = TextVectorization()




In [17]:
#layer.adapt(X_train.values)
vectorized_X_train = layer(X_train.values)
vectorized_X_test = layer(X_test.values)
print(vectorized_X_train)


tf.Tensor(
[[1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]], shape=(508000, 397), dtype=int64)


In [18]:
X_train.values

array(['please help tried launch game downloaded ofr freaking hours installed began clicking game message warrning comes like cannot find msvcp dll please install application anyone',
       'good game', 'nice', ..., 'gelly',
       'funny game really bad server shuts every night asia please ea purchase pubg bluehole provide us better server know ea destroyed many nice games studios even worst game ea made better optimization network pubg please ea time gain reputation back make whole life right decision xdand wanna apologize ubi calling servers potatoes ubi servers potatoes could use describe pubg change recommendation play game hours',
       'good concept bad execution years thank pubg game type play real battle royal games actually work'],
      dtype=object)

In [19]:
from tensorflow.keras import layers

model = tf.keras.models.Sequential()
model.add(layers.Embedding(max_features + 1, embedding_dim))
#model.layers[0].trainable = False
#model.add(layers.Dropout(0.05))
model.add(layers.GlobalAveragePooling1D())
#model.add(layers.Dropout(0.05))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

model.compile(loss="binary_crossentropy", optimizer='adam', metrics="accuracy")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          3200016   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 3,200,033
Trainable params: 3,200,033
Non-trainable params: 0
_________________________________________________________________


In [21]:
#train model
#epochs = 10
#history = model.fit(
#   vectorized_X_train,y_train,
#   epochs=epochs,batch_size=32)

Epoch 1/3
15875/15875 [==============================] - 531s 33ms/step - loss: 0.6617 - accuracy: 0.6317
Epoch 2/3
15875/15875 [==============================] - 535s 34ms/step - loss: 0.6616 - accuracy: 0.6320
Epoch 3/3
15875/15875 [==============================] - 524s 33ms/step - loss: 0.6616 - accuracy: 0.6318


In [22]:
# evaluate model
#model.evaluate(vectorized_X_test, y_test, batch_size=512)

249/249 [==============================] - 2s 7ms/step - loss: 0.6610 - accuracy: 0.6268


[0.660976231098175, 0.626758873462677]

In [ ]:
#BERT model

In [ ]:
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

In [ ]:


import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from time import time
import io
import re

import pickle
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten

from datetime import datetime



In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
class GameReviewData:
    DATA_COLUMN = "cleaned"
    LABEL_COLUMN = "recommend"

    def __init__(self, train, test, tokenizer, classes, max_seq_len=192):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes
    
        ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])

    def _prepare(self, df):
        x, y = [], []
    
        for _, row in tqdm(df.iterrows()):
            text, label = row[GameReviewData.DATA_COLUMN], row[GameReviewData.LABEL_COLUMN]
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            x.append(token_ids)
            y.append(self.classes.index(label))

        return np.array(x), np.array(y)

    def _pad(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)




In [ ]:
from sklearn.model_selection import train_test_split

train,test= train_test_split(df_balanced,
test_size=0.2)


In [ ]:
classes = df_balanced.recommend.unique().tolist()
classes

[1, 0]

In [ ]:
from tqdm import tqdm
processed_data= GameReviewData(train,test, tokenizer, [0,1], max_seq_len=50)

512748it [04:12, 2032.96it/s]
128188it [01:04, 2000.25it/s]


max seq_len 2661


In [ ]:
processed_data.train_x[0]

array([  101, 11030,  1403, 23078,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [ ]:
bert_layer = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(max_seq_len,train_layers):
    
  input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
  
   
  bert_inputs = [input_ids]
# Freeze layers, i.e. only train number of layers specified, starting from the top
    
  if not train_layers == -1:
        
    retrain_layers = []
    
    for retrain_layer_number in range(train_layers):

      layer_code = '_' + str(11 - retrain_layer_number)
      retrain_layers.append(layer_code)

    for w in bert_layer.weights:
      if not any([x in w.name for x in retrain_layers]):
              w._trainable = False

     # End of freezing section

  output         = bert_layer(bert_inputs)[0]
  print(output.shape)
  #cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
  #cls_out = keras.layers.Dropout(0.5)(cls_out)
  logits = keras.layers.Dense(units=256, activation="relu")(output)
  logits = keras.layers.Dropout(0.1)(logits)
  logits = keras.layers.Dense(units=1, activation="sigmoid")(logits)

  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))



  model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
                loss="binary_crossentropy",
                metrics=["accuracy"])

  model.summary()

  return model

In [ ]:
model = create_model(processed_data.max_seq_len,train_layers=-1)

(None, 50, 768)
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 50)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 50, 768), (None,  108310272 
_________________________________________________________________
dense_5 (Dense)              (None, 50, 256)           196864    
_________________________________________________________________
dropout_39 (Dropout)         (None, 50, 256)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 50, 1)             257       
Total params: 108,507,393
Trainable params: 108,507,393
Non-trainable params: 0
_________________________________________________________________


In [ ]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  1


In [ ]:
import datetime
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
log_dir = "log/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=processed_data.train_x, 
  y=processed_data.train_y,
  validation_split=0.1,
  batch_size=128,
  shuffle=True,
  epochs=8,
  callbacks=[tensorboard_callback]
)

Epoch 1/8
3606/3606 [==============================] - 1877s 520ms/step - loss: 0.6175 - accuracy: 0.6677 - val_loss: 0.6085 - val_accuracy: 0.6783
Epoch 2/8
3606/3606 [==============================] - 1865s 517ms/step - loss: 0.5991 - accuracy: 0.6831 - val_loss: 0.6058 - val_accuracy: 0.6811
Epoch 3/8
3606/3606 [==============================] - 1877s 521ms/step - loss: 0.5934 - accuracy: 0.6869 - val_loss: 0.5923 - val_accuracy: 0.6875
Epoch 4/8
3606/3606 [==============================] - 1883s 522ms/step - loss: 0.5899 - accuracy: 0.6892 - val_loss: 0.5847 - val_accuracy: 0.6911
Epoch 5/8
3606/3606 [==============================] - 1883s 522ms/step - loss: 0.5872 - accuracy: 0.6911 - val_loss: 0.5790 - val_accuracy: 0.6937
Epoch 6/8
3606/3606 [==============================] - 1885s 523ms/step - loss: 0.5853 - accuracy: 0.6925 - val_loss: 0.5891 - val_accuracy: 0.6919
Epoch 7/8
3481/3606 [===========================>..] - ETA: 59s - loss: 0.5834 - accuracy: 0.6940

In [ ]:
model.evaluate(processed_data.test_x,processed_data.test_y,batch_size=1048)

  1/401 [..............................] - ETA: 0s - loss: 0.5758 - accuracy: 0.7017

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir log

<IPython.core.display.Javascript object>